# **Sistema de Gestión de Servicios y Pasajes – ArgentinaTur**

In [ ]:
from datetime import datetime, date
from abc import ABC
from typing import List

# Medio de Pago (abstracta)
class MedioPago(ABC):
    pass

# Tarjeta de Crédito
class TarjetaCredito(MedioPago):
    def __init__(self, numero, dni_titular, nombre, fecha_vencimiento):
        self.__numero = numero
        self.__dni_titular = dni_titular
        self.__nombre = nombre
        self.__fecha_vencimiento = fecha_vencimiento

# MercadoPago
class MercadoPago(MedioPago):
    def __init__(self, celular, email):
        self.__celular = celular
        self.__email = email

# Ualá
class Uala(MedioPago):
    def __init__(self, email, nombre_titular):
        self.__email = email
        self.__nombre_titular = nombre_titular


# Asiento
class Asiento:
    def __init__(self, numero):
        self.__numero = numero
        self.__ocupado = False # Se crean desocupados

    def set_asiento_ocupado(self,estado):
        self.__ocupado = estado

    def esta_ocupado(self):
        return self.__ocupado
    
    def mostrar_asiento(self):
        print(f"El asiento {self.__numero} está {'No Disponible' if self.__ocupado else 'Disponible'}") # Imprime si esta disponible o no


# Unidad (Colectivo)
class Unidad:
    def __init__(self, patente, cantidad_asientos):
        self.__patente = patente
        self.__asientos = [Asiento(i + 1) for i in range(cantidad_asientos)] # inicializar el map con todos los asientos (desocupados)
    
    def ver_asientos_disponibles(self):
        disponibles = []
        for asiento in self.__asientos:
            if not asiento.esta_ocupado():
                disponibles.append(asiento)
        disponibles.mostrar_asiento()

    def ocupar_asiento(self, numero_asiento):
        self.__asientos[numero_asiento].set_asiento_ocupado(True)

    def obtener_asiento(self, numero):
        return self.__asientos[numero]


# Pasajero : Podria ser un sruct
class Pasajero:
    def __init__(self, nombre, email, dni):
        self.__nombre = nombre
        self.__email = email
        self.__dni = dni


# Reserva
class Reserva:
    def __init__(self, fecha_hora, pasajeros:list[Pasajero], asientos:list[Asiento]):
        self.__fecha_hora = fecha_hora
        self.__pasajeros = pasajeros
        self.__asientos = asientos
        self.__confirmada = False
        self.__pasajero_cabecera = self.__pasajeros[1] #es el pasajero que va a figurar como titular de la reserva (se aclara a la hora de reservar que es el 1ro que se registra)   
        for a in self.__asientos:
            a.set_asiento_ocupado(True)

    def obtener_estado(self):
        return self.__confirmada
    
    def obtener_pas_cabecera(self):
        return self.__pasajero_cabecera

    def confirmar_reserva(self):
        self.__confirmada = True

    def cancelar_reserva_si_no_esta_confirmada(self):
        if not (self.__confirmada):
            for asiento in self.__asientos:
                asiento.set_asiento_ocupado(False)

    def mostrar_reserva(self):
        pass # completar imprimiendo en pantalla los datos de la reserva
        

# Venta
class Venta:
    def __init__(self, reserva, medio_pago : MedioPago):
        self.__reserva = reserva
        self.__medio_pago = medio_pago
        self.__concretada = False
    
        self.__pagar(reserva, medio_pago)

    def __pagar(self, reserva, medio_pago: MedioPago):
        # aca va algo como pagado = medio_pago.se_debito_correctamente()
        pagado = True
        if(pagado):
            #se concreto la venta
            self.__reserva.confirmar_reserva()
            self.__concretada = True
        else:
            print("Error en el metodo de pago !!! Reintentar mas tarde")

    def obtener_estado_venta(self):
        return self.__concretada



class ItinerarioManager:
    __instance = None

    # Singleton
    def __new__(cls):
        if cls.__instance is None:
            cls.__instance = super(ItinerarioManager, cls).__new__(cls)
            cls.__instance.__itinerarios = {}
        return cls.__instance
    
    def crear_itinerario(self):
        # Inicializar lista de ciudades del itinerario
        ciudades_nuevo_it = []

        # Ingresar Nombre de nuevo itinerario
        itinerario_nombre = str(input("Ingrese el nombre de Itinerario a crear: "))

        # Ingresar Ciudad de Origen
        cod = int(input("Introduce el código de la ciudad de origen: "))
        nombre = str(input("Introduce el nombre de la ciudad de origen: "))
        provincia = str(input("Introduce el nombre de la provincia de la ciudad de origen: "))

        # Agregar ciudad de origen
        ciudades_nuevo_it.append(Ciudad(cod, nombre, provincia))
        
        # Ingresar ciudades parada
        cod = int(input("Introduce código de la ciudad de parada o -1 para terminar: "))
        while(cod > 0):  
            nombre = str(input("Introduce el nombre de la ciudad de parada : "))
            provincia = str(input("Introduce el nombre de la provincia de la ciudad de parada: "))
            ciudades_nuevo_it.append(Ciudad(cod, nombre, provincia))
            # Agregar ciudad de parada 
            cod = int(input("Introduce código de la ciudad de parada o -1 para terminar: "))

        # Ingresar ciudad de Destino
        cod = int(input("Introduce el código de la ciudad destino: "))
        nombre = str(input("Introduce el nombre de la ciudad destino: "))
        provincia = str(input("Introduce el nombre de la provincia de la ciudad destino: "))

        # Agregar ciudad de destino
        ciudades_nuevo_it.append(Ciudad(cod, nombre, provincia))

        # Crear itinerario
        nuevo_itinerario = Itinerario(itinerario_nombre, ciudades_nuevo_it)

        # Agregar el itinerario
        self.agregar_itinerario(nuevo_itinerario)

    def agregar_itinerario(self, itinerario):
        nombre = itinerario.obtener_nombre()
        if nombre in self.__itinerarios:
            print(f"Error el itinerario '{nombre}' ya existe.")
        else:
            self.__itinerarios[nombre] = itinerario

    def obtener_itinerario(self, nombre_itinerario):
        return self.__itinerarios[nombre_itinerario]
    
# Itinerario
class Itinerario:
    def __init__(self, nombre, ciudades: list['Ciudad']):
        self.__nombre = nombre
        self.__ciudades = ciudades

    def obtener_nombre(self):
        return self.__nombre

    def mostrar_itinerario(self):
        print("Itinerario: ", self.__nombre)
        print(f"    Ciudad de origen:  {self.__ciudades[0].obtener_info()}")

        if len(self.__ciudades) > 2: # Si hay ciudades de parada las mostramos
            print("    Paradas: ")
            contador = 1
            for ciudad in self.__ciudades[1:-1]:  # Slicing para iterar desde el segundo hasta el penúltimo elemento
                print(f"                [{contador}]:  ",ciudad.obtener_info())
                contador += 1
        
        print(f"    Ciudad de destino: {self.__ciudades[-1].obtener_info()}")


# Ciudad
class Ciudad:
    def __init__(self, codigo, nombre, provincia):
        self.__codigo = codigo
        self.__nombre = nombre
        self.__provincia = provincia

    def obtener_info(self):
        return f"{self.__codigo} - {self.__nombre} / {self.__provincia}"

    def obtener_nombre(self):
        return self.__nombre
    
    def obtener_codigo(self):
        return self.__codigo
    
    def obtener_provincia(self):
        return self.__provincia

# Servicio
class Servicio:
    def __init__(self, nombre : str, unidad, fecha_partida, fecha_llegada, calidad, precio, itinerario):
        self.__nombre = nombre
        self.__unidad = unidad
        self.__fecha_partida = fecha_partida
        self.__fecha_llegada = fecha_llegada
        self.__calidad = calidad
        self.__precio = precio
        self.__itinerario = itinerario
        self.__disponible = True
        self.__reservas = {}
      
    def cancelar_reservas_no_confirmada(self): # Funcion que cancela las reservas no confirmadas del servicio acutal, se debera llamar cuando quede media hora para el viaje
        for reserva_actual in self.__reservas.values:
            reserva_actual.cancelar_reserva_si_no_esta_confirmada()
            
    def mostrar_servicio(self): # Muestra el servicio
        if(self.__disponible==True):
            print(f"Servicio: {self.__nombre}")
            print(f"Calidad: {self.__calidad}")
            print(f"Fecha de partida: {self.__fecha_partida}")
            print(f"fecha de llegada: {self.__fecha_llegada}")
            self.__itinerario.mostrar_itinerario()

    # Añade una reserva
    def aniadir_reserva(self, numero_reserva, pasajeros, numero_asientos, fecha_hora):
        nueva_reserva = Reserva(fecha_hora, pasajeros, self.__unidad.obtener_asientos(numero_asientos)) # Le paso el asiento de la unidad asi reserva tiene una ref a la misma instancia
        if nueva_reserva in self.__reservas:
            print(f"Error la reserva nro: '{numero_reserva}' ya existe.")
        else:
            self.__reservas[numero_reserva] = nueva_reserva

    def obtener_numero_de_reserva(self):
        return len(self.__reservas) + 1 # Obtengo el numero de reserva disponible para crear, si tenia 5 reservar, si creo una el numero sera 6
    

    # añadir mostrar Unidad y asientos


# Argentur
class Argentur:
    def __init__(self, sistema_activo):
        self.__sistema_activo = sistema_activo
        self.__itinerarioManager = ItinerarioManager()
        self.__reservaManager = ReservaManager()
        self.__servicios = {}
        self.__ventas = [] # Transformar ventas en map y acceder segun su numero talvez?
        self.__reservas = {}

    # Itinerario
    def crear_itinerario(self):
        self.__itinerarioManager.crear_itinerario()

    def crear_itinerario_por_defecto(self, itinerario: Itinerario):
        self.__itinerarioManager.agregar_itinerario(itinerario)

    def obtener_itinerario_por_nombre(self, nombre_it):
        return self.__itinerarioManager.obtener_itinerario(nombre_it)

    # Servicios
    def consultar_servicio_disponibles(self):
        for servicio in self.__servicios.values():
            servicio.mostrar_servicio()

    def agregar_servicio(self,nombre,unidad, fecha_partida, fecha_llegada, calidad, precio, itinerario):
        servicio = Servicio(nombre, unidad, fecha_partida, fecha_llegada, calidad, precio, itinerario)
#        if nombre in self.__servicios:
#            print(f"Error el servicio '{nombre}' ya existe.")
#        else:
#            self.__servicios[nombre] = servicio
        self.__servicios[nombre] = servicio # Si el servicio ya existe lo reemplaza, sino lo agrega

    def concretar_venta(self):
        #completar
        pass

    def agregar_venta(self,reserva, medio_pago): # Solo agrega la venta
        venta = Venta(reserva, medio_pago)
        self.__ventas.append(venta)

    # Reserva metodo privado , luego hacer el metodo para ingresar por consola los datos del pasajero y mostrar los asientos disponibles, etc
    def agregar_reserva(self, nombre_servicio, numeros_asientos:list[int], nombres_pasajeros:list[str], email_pasajeros:list[str], dni_pasajeros:list[int]):
        a=0

        #chequeo de asientos
        
        for serv in self.__servicios:
            if (nombre_servicio ==  self.__servicios[serv].verNombreServicio()): # no ni idea ahi reviso
                servicio_seleccionado = self.__servicios[serv]
                a=1
                break
        if(not(a)):
            ValueError("Servicio no existente")
        self.__reservaManager.__agregar_reserva_manag(servicio_seleccionado,numeros_asientos, nombres_pasajeros,email_pasajeros,dni_pasajeros)
        
        

class ReservaManager:
    def __agregar_reserva_manag(self,servicio_seleccionado,numeros_asientos:list[int], nombres_pasajeros:list[str], email_pasajeros:list[str], dni_pasajeros:list[str]):
        numero_reserva = servicio_seleccionado.obtener_numero_de_reserva() 
        fecha_hora = datetime.now()
        pasajeros = []
        for pas in nombres_pasajeros:
            pasajeros.append(Pasajero(nombres_pasajeros[pas], email_pasajeros[pas], dni_pasajeros[pas]))

        servicio_seleccionado.aniadir_reserva(numero_reserva, pasajeros, numeros_asientos, fecha_hora)

    

# Punto de entrada al programa

In [10]:

Sistema_Argentur = Argentur(True)

calidad_economico = "Económico"
calidad_premium = "Premium"

# Crear Itinerarios predefinidos
santoTome = Ciudad(3016, "Santo Tome", "Santa Fe")
santaFe = Ciudad(3000, "Santa Fe", "Santa Fe")
parana = Ciudad(3100,"Parana","Entre Rios")

#           Salida   ,Parada 1, Destino
ciudades1 = [santoTome, santaFe, parana]
ciudades2 = [parana, santaFe, santoTome]
itinerario1 = Itinerario("Viaje a Parana",ciudades1)
itinerario2 = Itinerario("Viaje a Santo Tome",ciudades2)

Sistema_Argentur.crear_itinerario_por_defecto(itinerario1)
Sistema_Argentur.crear_itinerario_por_defecto(itinerario2)

# prueba para mostrar itinerarios (borrar)
itinerarioInstanceTest1 = Sistema_Argentur.obtener_itinerario_por_nombre("Viaje a Parana")
itinerarioInstanceTest2 = Sistema_Argentur.obtener_itinerario_por_nombre("Viaje a Santo Tome")
#itinerarioInstanceTest1.mostrar_itinerario()
#itinerarioInstanceTest2.mostrar_itinerario()

unidad1 = Unidad("unidad1", 10)
fecha1 = datetime(2025, 4, 27, 12, 30)
fecha2 = datetime(2025, 4, 27, 12, 30)

# Crear Servicios
servicio1 = "Servicio 1"
servicio2 = "Servicio 2"
                                    #nombre,unidad, fecha_partida, fecha_llegada, calidad, precio, itinerario
Sistema_Argentur.agregar_servicio(servicio1, unidad1, fecha1, fecha2, calidad_economico, 1000, itinerarioInstanceTest1)
Sistema_Argentur.agregar_servicio(servicio2, unidad1, fecha1, fecha2, calidad_premium, 1500, itinerarioInstanceTest2)

Sistema_Argentur.consultar_servicio_disponibles()

# Crear Pasajeros
num_asientos = [1, 2, 3] # Asientos a reservar
pasajeros_nombre = ["Tomi", "Hernan", "Maxi"]
pasajeros_mail = ["salame@gmail.com", "salame1@gmail.com", "salame2@gmail.com"]
pasajeros_dni = [12345678, 12345679, 12345680]
                #agregar_reserva(nombre_servicio, numeros_asientos:list[int], nombres_pasajeros:list[str], email_pasajeros:list[str], dni_pasajeros:list[int]
#Sistema_Argentur.agregar_reserva("Servicio 1", [1, 2, 3], ["Tomi", "Hernan", "Maxi" ], ["salame@gmail.com", "salame1@gmail.com", "salame2@gmail.com"], [12345678, 12345679, 12345680])
Sistema_Argentur.agregar_reserva("Servicio 1", num_asientos,pasajeros_nombre, pasajeros_mail, pasajeros_dni)


Error el itinerario 'Viaje a Parana' ya existe.
Error el itinerario 'Viaje a Santo Tome' ya existe.
Servicio: Servicio 1
Calidad: Económico
Fecha de partida: 2025-04-27 12:30:00
fecha de llegada: 2025-04-27 12:30:00
Itinerario:  Viaje a Parana
    Ciudad de origen:  3016 - Santo Tome / Santa Fe
    Paradas: 
                [1]:   3000 - Santa Fe / Santa Fe
    Ciudad de destino: 3100 - Parana / Entre Rios
Servicio: Servicio 2
Calidad: Premium
Fecha de partida: 2025-04-27 12:30:00
fecha de llegada: 2025-04-27 12:30:00
Itinerario:  Viaje a Santo Tome
    Ciudad de origen:  3100 - Parana / Entre Rios
    Paradas: 
                [1]:   3000 - Santa Fe / Santa Fe
    Ciudad de destino: 3016 - Santo Tome / Santa Fe


AttributeError: 'Servicio' object has no attribute 'verNombreServicio'